In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import folium
import numpy as np
import seaborn as sns
import os
import fastparquet
import warnings
import geopy
from geopy.point import Point
import time
from pandas.core.common import SettingWithCopyWarning
import plotly.graph_objects as go
import plotly.express as px
from datetime import datetime, timedelta, date
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [2]:
# Load in price data
occ_data = pd.read_parquet('C:/Users/mattg/Desktop/Hobbies/airbnb_reports/bucket_data/occupancy')
occ_data.id = occ_data.id.astype('str')
miami_example = occ_data[occ_data['id'] == '52364652.0']
#start_time = time.time()
#miami_example['pulled'] = pd.to_datetime(miami_example['pulled'])
#print('done with datatype change')
#middle_time = time.time()
#miami_example['pulled'] = miami_example['pulled'].dt.date
#print('done with change to date')
#end_time = time.time()

#print('Start Time '+ str(start_time))
#print('Middle Time '+ str(middle_time-start_time))
#print('End Time '+ str(end_time-start_time))


In [3]:
miami_example_by_pull_time = miami_example.groupby(['id','pulled'])['available','min_nights','available_for_checkin','available_for_checkout','bookable'].mean().reset_index()
miami_example_by_pull_time_count = miami_example.groupby(['pulled','date'])['available','min_nights','available_for_checkin','available_for_checkout','bookable'].count().reset_index()

# Create traces
miami_example_grouped_fig = go.Figure()
miami_example_grouped_fig.add_trace(go.Scatter(x=miami_example_by_pull_time['pulled'], y=miami_example_by_pull_time['available'],
                    mode='lines',
                    name='Available'))
miami_example_grouped_fig.add_trace(go.Scatter(x=miami_example_by_pull_time['pulled'], y=miami_example_by_pull_time['available_for_checkin'],
                    mode='lines', 
                    name='Available for Check In'))
miami_example_grouped_fig.add_trace(go.Scatter(x=miami_example_by_pull_time['pulled'], y=miami_example_by_pull_time['available_for_checkout'],
                    mode='lines', 
                    name='Available for Check Out'))
miami_example_grouped_fig.add_trace(go.Scatter(x=miami_example_by_pull_time['pulled'], y=miami_example_by_pull_time['bookable'],
                    mode='lines', 
                    name='Bookable'))
miami_example_grouped_fig.update_layout(title_text="Occupancy Data by Date", xaxis_title='Date', yaxis_title='Availability')
filename = "newsletter_features/occupancy_data_line_fig_miami_example.png"
miami_example_grouped_fig.write_image(filename, engine='kaleido', width=2000, height=500)
miami_example_grouped_fig.show()

In [21]:
days_to_subtract = 30
last_day_from = pd.to_datetime('05/31/2022')
date_30_days_ago = last_day_from - timedelta(days=days_to_subtract)
occ_data['pulled'] = pd.to_datetime(occ_data.pulled)
pull_time_old_start = pd.to_datetime('19/04/2022')
pull_time_old_end = pd.to_datetime('21/04/2022')
pull_time_current_start = pd.to_datetime('04/05/2022')
pull_time_current_end = pd.to_datetime('06/05/2022')


occ_data_last30 = occ_data[(occ_data['date'] > str(date_30_days_ago)) & (occ_data['date'] <= str(last_day_from))]
print(occ_data_last30.shape)
#price_data_last30_avgs = price_data_last30.groupby(['id','pull_time']).mean().reset_index()
occ_data_previous_pull = occ_data_last30[(occ_data_last30['pulled'] > str(pull_time_old_start)) & (occ_data_last30['pulled'] < str(pull_time_old_end))]
print(occ_data_previous_pull.shape)
occ_data_current_pull = occ_data_last30[(occ_data_last30['pulled'] > str(pull_time_current_start)) & (occ_data_last30['pulled'] < str(pull_time_current_end))]
print(occ_data_current_pull.shape)
occ_data_previous_pull_avgs = occ_data_previous_pull.groupby('id').mean().reset_index()
print(occ_data_previous_pull_avgs.shape)
occ_data_current_pull_avgs = occ_data_current_pull.groupby('id').mean().reset_index()
print(occ_data_current_pull_avgs.shape)
occ_trend = occ_data_previous_pull_avgs.merge(occ_data_current_pull_avgs, how='inner', on='id')
print(occ_trend.shape)


C:\Users\mattg\AppData\Local\Temp\ipykernel_3312\2004464441.py:5: UserWarning:

Parsing '19/04/2022' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.

C:\Users\mattg\AppData\Local\Temp\ipykernel_3312\2004464441.py:6: UserWarning:

Parsing '21/04/2022' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.



(3733530, 11)
(152730, 11)
(2086890, 11)
(1681, 8)
(5608, 8)
(1681, 15)


0          2022-04-19 06:22:01
1          2022-04-19 06:22:01
2          2022-04-19 06:22:01
3          2022-04-19 06:22:01
4          2022-04-19 06:22:01
                   ...        
45424610   2022-05-18 06:22:11
45424611   2022-05-18 06:22:11
45424612   2022-05-18 06:22:11
45424613   2022-05-18 06:22:11
45424614   2022-05-18 06:22:11
Name: pulled, Length: 45424615, dtype: datetime64[ns]

In [22]:
# Look at occ data in head(10) statement above to figure out what delta calcs need to be made
occ_trend['available_delta'] = occ_trend['available_y'] - occ_trend['available_x']
occ_trend['available_for_checkin_delta'] = occ_trend['available_for_checkin_y'] - occ_trend['available_for_checkin_x']
occ_trend['available_for_checkout_delta'] = occ_trend['available_for_checkout_y'] - occ_trend['available_for_checkout_x']
occ_trend['bookable_delta'] = occ_trend['bookable_y'] - occ_trend['bookable_x']
occ_trend['available_delta_pct'] = occ_trend['available_delta']/occ_trend['available_x']*100
occ_trend['available_for_checkin_delta_pct'] = occ_trend['available_for_checkin_delta']/occ_trend['available_for_checkin_x']*100
occ_trend['available_for_checkout_delta_pct'] = occ_trend['available_for_checkout_delta']/occ_trend['available_for_checkout_x']*100
occ_trend['bookable_delta_pct'] = occ_trend['bookable_delta']/occ_trend['bookable_x']*100

occ_data_w_listing_loc = pd.read_csv('post_mort_and_roi_calcs_june19.csv')
occ_data_w_listing_loc = occ_data_w_listing_loc[['id','zipcode','City','Metro','CountyName','State']]
occ_trend['id'] = occ_trend['id'].astype('float64')
occ_trend_wloc = occ_trend.merge(occ_data_w_listing_loc, how='inner', on='id')

In [23]:
florida_occ_trends = occ_trend_wloc[occ_trend_wloc['State'] == 'FL']
florida_occ_trends_zip = florida_occ_trends.groupby('zipcode').mean(['available_delta_pct','available_for_checkin_delta_pct','available_for_checkout_delta_pct','bookable_delta_pct']).reset_index()
florida_occ_trends_city = florida_occ_trends.groupby('City').mean(['available_delta_pct','available_for_checkin_delta_pct','available_for_checkout_delta_pct','bookable_delta_pct']).reset_index()
"""
florida_price_trends_fig = go.Figure()
florida_price_trends_fig.add_trace(go.Scatter(x=price_data_date['check_in'], y=price_data_date['cleaning_fee'],
                    mode='lines',
                    name='Cleaning Fee'))
florida_price_trends_fig.add_trace(go.Scatter(x=price_data_date['check_in'], y=price_data_date['service_fee'],
                    mode='lines',
                    name='Service Fee'))
florida_price_trends_fig.add_trace(go.Scatter(x=price_data_date['check_in'], y=price_data_date['total_price'],
                    mode='lines', 
                    name='Total Price'))
florida_price_trends_fig.update_layout(title_text="Pricing Data by Pull Time", xaxis_title='Date', yaxis_title='Price ($)')
filename = "newsletter_features/florida_price_trends_fig_june19.png"
florida_price_trends_fig.write_image(filename, engine='kaleido', width=2000, height=500)
florida_price_trends_fig.show()
"""
import plotly.graph_objects as go

zips = florida_occ_trends_zip['zipcode']
vals = [florida_occ_trends_zip['available_delta_pct'], florida_occ_trends_zip['available_for_checkin_delta_pct'], florida_occ_trends_zip['available_for_checkout_delta_pct'], florida_occ_trends_zip['bookable_delta_pct']]
font_color = ['rgb(40,40,40)'] +  [['rgb(255,0,0)' if v < 0 else 'rgb(0,125,0)' for v in vals[k]] for k in range(4)]

table_trace = go.Table(
                 columnwidth= [50]+[50]+[50]+[50]+[50],
                 columnorder=[0, 1, 2, 3, 4],
                 header = dict(height = 40,
                               values = [['<b>Zip Code</b>'], ['<b>Availability</b>'], ['<b>Check-in Availability</b>'], ['<b>Check-out Availability</b>'],['<b>Bookable</b>']],
                               line = dict(color='rgb(50,50,50)'),
                               align = ['left']*5,
                               font = dict(color=['rgb(45,45,45)']*4, size=14),
                             
                              ),
                 cells = dict(values = [zips, vals[0], vals[1], vals[2], vals[3]],
                              line = dict(color='#506784'),
                              align = ['left']*5,
                              
                              font = dict(family="Arial", size=14, color=font_color),
                              format = [None, ",.2f"],  #add % sign here
                              height = 30,
                              fill = dict(color='rgb(245,245,245)'))
                             )
                 

layout = go.Layout(width=850, height=650, autosize=False, 
              title_text='Recent Occupancy Trends in Miami (%) (M/M)',
                   title_x=0.5, showlegend=False)
florda_occ_trend_fig = go.Figure(data=[table_trace], layout=layout)
filename = "newsletter_features/florida_occ_trends_table_june19.png"
florda_occ_trend_fig.write_image(filename, engine='kaleido', width=875, height=700)
florda_occ_trend_fig.show()

# Look to do one price value and look and price change by day of the week

In [24]:
fwd_looking_days = 28
pull_time_current = pd.to_datetime('05/05/2022') # Would be set as date for "today" in the future
fwd_looking_date_current = pull_time_current + timedelta(days=fwd_looking_days)
pull_time_old = last_day_from - timedelta(days=7)
fwd_looking_date_old = pull_time_old + timedelta(days=fwd_looking_days)

occ_data_last30_current = occ_data[(occ_data['date'] > str(pull_time_current)) & (occ_data['date'] <= str(fwd_looking_date_current))]
occ_data_last30_old = occ_data[(occ_data['date'] > str(pull_time_old)) & (occ_data['date'] <= str(fwd_looking_date_old))]
print(occ_data_last30_current.shape)
print(occ_data_last30_old.shape)
occ_data_previous_pull_avgs = occ_data_last30_current.groupby('id').mean().reset_index()
print(occ_data_previous_pull_avgs.shape)
occ_data_current_pull_avgs = occ_data_last30_old.groupby('id').mean().reset_index()
print(occ_data_current_pull_avgs.shape)
occ_trend = occ_data_previous_pull_avgs.merge(occ_data_current_pull_avgs, how='inner', on='id')
print(occ_trend.shape)

(3484628, 11)
(3484628, 11)
(5608, 8)
(5608, 8)
(5608, 15)


In [39]:
#occ_trend['available_delta'] = occ_trend['available_y'] - occ_trend['available_x']
#occ_trend['available_for_checkin_delta'] = occ_trend['available_for_checkin_y'] - occ_trend['available_for_checkin_x']
#occ_trend['available_for_checkout_delta'] = occ_trend['available_for_checkout_y'] - occ_trend['available_for_checkout_x']
#occ_trend['bookable_delta'] = occ_trend['bookable_y'] - occ_trend['bookable_x']
#occ_trend['available_delta_pct'] = occ_trend['available_delta']/occ_trend['available_x']*100
#occ_trend['available_for_checkin_delta_pct'] = occ_trend['available_for_checkin_delta']/occ_trend['available_for_checkin_x']*100
#occ_trend['available_for_checkout_delta_pct'] = occ_trend['available_for_checkout_delta']/occ_trend['available_for_checkout_x']*100
#occ_trend['bookable_delta_pct'] = occ_trend['bookable_delta']/occ_trend['bookable_x']*100
occ_trend['available_delta_pct'] = (occ_trend['available_y'] - occ_trend['available_x']) * 100
occ_trend['available_for_checkin_delta_pct'] = (occ_trend['available_for_checkin_y'] - occ_trend['available_for_checkin_x']) * 100
occ_trend['available_for_checkout_delta_pct'] = (occ_trend['available_for_checkout_y'] - occ_trend['available_for_checkout_x']) * 100
occ_trend['bookable_delta_pct'] = (occ_trend['bookable_y'] - occ_trend['bookable_x']) * 100
data_w_listing_loc = pd.read_csv('post_mort_and_roi_calcs_june19.csv')
data_w_listing_loc = data_w_listing_loc[['id','zipcode','City','Metro','CountyName','State']]
occ_trend['id'] = occ_trend['id'].astype('float64')
occ_trend_wloc = occ_trend.merge(data_w_listing_loc, how='inner', on='id')

In [42]:
florida_occ_trends = occ_trend_wloc[occ_trend_wloc['State'] == 'FL']
florida_occ_trends_zip = florida_occ_trends.groupby('zipcode').mean(['available_delta_pct','available_for_checkin_delta_pct','available_for_checkout_delta_pct','bookable_delta_pct']).reset_index()
florida_occ_trends_city = florida_occ_trends.groupby('City').mean(['available_delta_pct','available_for_checkin_delta_pct','available_for_checkout_delta_pct','bookable_delta_pct']).reset_index()
"""
florida_price_trends_fig = go.Figure()
florida_price_trends_fig.add_trace(go.Scatter(x=price_data_date['check_in'], y=price_data_date['cleaning_fee'],
                    mode='lines',
                    name='Cleaning Fee'))
florida_price_trends_fig.add_trace(go.Scatter(x=price_data_date['check_in'], y=price_data_date['service_fee'],
                    mode='lines',
                    name='Service Fee'))
florida_price_trends_fig.add_trace(go.Scatter(x=price_data_date['check_in'], y=price_data_date['total_price'],
                    mode='lines', 
                    name='Total Price'))
florida_price_trends_fig.update_layout(title_text="Pricing Data by Pull Time", xaxis_title='Date', yaxis_title='Price ($)')
filename = "newsletter_features/florida_price_trends_fig_june19.png"
florida_price_trends_fig.write_image(filename, engine='kaleido', width=2000, height=500)
florida_price_trends_fig.show()
"""
import plotly.graph_objects as go

zips = florida_occ_trends_zip['zipcode']
vals = [florida_occ_trends_zip['available_delta_pct'], florida_occ_trends_zip['available_for_checkin_delta_pct'], florida_occ_trends_zip['available_for_checkout_delta_pct'], florida_occ_trends_zip['bookable_delta_pct']]
font_color = ['rgb(40,40,40)'] +  [['rgb(0,125,0)' if v < 0 else 'rgb(255,0,0)' for v in vals[k]] for k in range(4)]

table_trace = go.Table(
                 columnwidth= [50]+[50]+[50]+[50]+[50],
                 columnorder=[0, 1, 2, 3, 4],
                 header = dict(height = 40,
                               values = [['<b>Zip Code</b>'], ['<b>Availability</b>'], ['<b>Check-in Availability</b>'], ['<b>Check-out Availability</b>'],['<b>Bookable</b>']],
                               line = dict(color='rgb(50,50,50)'),
                               align = ['left']*5,
                               font = dict(color=['rgb(45,45,45)']*4, size=14),
                             
                              ),
                 cells = dict(values = [zips, vals[0], vals[1], vals[2], vals[3]],
                              line = dict(color='#506784'),
                              align = ['left']*5,
                              
                              font = dict(family="Arial", size=14, color=font_color),
                              format = [None, ",.2f"],  #add % sign here
                              height = 30,
                              fill = dict(color='rgb(245,245,245)'))
                             )
                 

layout = go.Layout(width=850, height=650, autosize=False, 
              title_text='Recent Occupancy Trends in Miami (%) (W/W)',
                   title_x=0.5, showlegend=False)
florda_occ_trend_fig = go.Figure(data=[table_trace], layout=layout)
filename = "newsletter_features/florida_occ_trends_table_weekly.png"
florda_occ_trend_fig.write_image(filename, engine='kaleido', width=875, height=700)
florda_occ_trend_fig.show()

# Look to do one price value and look and price change by day of the week

In [46]:
florida_occ_trends_zip_propcounts = florida_occ_trends.groupby('zipcode').count().reset_index()
florida_occ_trends_zip_propcounts

,zipcode,id,index_x,available_x,max_nights_x,min_nights_x,available_for_checkin_x,available_for_checkout_x,bookable_x,index_y,...,available_for_checkout_delta,bookable_delta,available_delta_pct,available_for_checkin_delta_pct,available_for_checkout_delta_pct,bookable_delta_pct,City,Metro,CountyName,State
0,33109,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
1,33125,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2,33127,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
3,33131,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
4,33133,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
5,33137,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
6,33138,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3
7,33139,13,13,13,13,13,13,13,13,13,...,13,13,13,13,13,13,13,13,13,13
8,33140,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
9,33141,6,6,6,6,6,6,6,6,6,...,6,6,6,6,6,6,6,6,6,6


In [48]:
florida_occ_trends = occ_trend_wloc[occ_trend_wloc['State'] == 'FL']
#florida_occ_trends_zip = florida_occ_trends.groupby('zipcode').mean(['available_delta_pct','available_for_checkin_delta_pct','available_for_checkout_delta_pct','bookable_delta_pct']).reset_index()
florida_occ_trends_city = florida_occ_trends.groupby('City').mean(['available_delta_pct','available_for_checkin_delta_pct','available_for_checkout_delta_pct','bookable_delta_pct']).reset_index()
"""
florida_price_trends_fig = go.Figure()
florida_price_trends_fig.add_trace(go.Scatter(x=price_data_date['check_in'], y=price_data_date['cleaning_fee'],
                    mode='lines',
                    name='Cleaning Fee'))
florida_price_trends_fig.add_trace(go.Scatter(x=price_data_date['check_in'], y=price_data_date['service_fee'],
                    mode='lines',
                    name='Service Fee'))
florida_price_trends_fig.add_trace(go.Scatter(x=price_data_date['check_in'], y=price_data_date['total_price'],
                    mode='lines', 
                    name='Total Price'))
florida_price_trends_fig.update_layout(title_text="Pricing Data by Pull Time", xaxis_title='Date', yaxis_title='Price ($)')
filename = "newsletter_features/florida_price_trends_fig_june19.png"
florida_price_trends_fig.write_image(filename, engine='kaleido', width=2000, height=500)
florida_price_trends_fig.show()
"""
import plotly.graph_objects as go

cities = florida_occ_trends_city['City']
vals = [florida_occ_trends_city['available_delta_pct'], florida_occ_trends_city['available_for_checkin_delta_pct'], florida_occ_trends_city['available_for_checkout_delta_pct'], florida_occ_trends_city['bookable_delta_pct']]
font_color = ['rgb(40,40,40)'] +  [['rgb(0,125,0)' if v < 0 else 'rgb(255,0,0)' for v in vals[k]] for k in range(4)]

table_trace = go.Table(
                 columnwidth= [50]+[50]+[50]+[50]+[50],
                 columnorder=[0, 1, 2, 3, 4],
                 header = dict(height = 40,
                               values = [['<b>City</b>'], ['<b>Availability</b>'], ['<b>Check-in Availability</b>'], ['<b>Check-out Availability</b>'],['<b>Bookable</b>']],
                               line = dict(color='rgb(50,50,50)'),
                               align = ['left']*5,
                               font = dict(color=['rgb(45,45,45)']*4, size=14),
                             
                              ),
                 cells = dict(values = [cities, vals[0], vals[1], vals[2], vals[3]],
                              line = dict(color='#506784'),
                              align = ['left']*5,
                              
                              font = dict(family="Arial", size=14, color=font_color),
                              format = [None, ",.2f"],  #add % sign here
                              height = 30,
                              fill = dict(color='rgb(245,245,245)'))
                             )
                 

layout = go.Layout(width=850, height=650, autosize=False, 
              title_text='Recent Occupancy Trends in Miami (%) (W/W)',
                   title_x=0.5, showlegend=False)
florda_occ_trend_fig = go.Figure(data=[table_trace], layout=layout)
filename = "newsletter_features/florida_occ_trends_table_city_weekly.png"
florda_occ_trend_fig.write_image(filename, engine='kaleido', width=875, height=700)
florda_occ_trend_fig.show()

# Look to do one price value and look and price change by day of the week